In [1]:
import numpy as np

# from sklearn.preprocessing import StandardScaler

In [2]:
while True:
# input file name
    print("Please enter file name")
    file_name = input()
    # file_name = "CS205_small_Data__30.txt"
    # file_name = "CS205_small_Data__19.txt"
    # file_name = "CS205_large_Data__34.txt"
    # file_name = "CS205_large_Data__6.txt"

# open and read
    try:
        with open(file_name, 'r') as file:
            file_content = file.readlines()
        break
    except FileNotFoundError:
        print("not fine")

data = []
    
for line in file_content:
    values = list(map(float, line.split()))
    data.append(values)
    
# change to numpy
data_array = np.array(data)
    
# split into label and feature
label_set = data_array[:, 0]    # 第一列
features_set = data_array[:, 1:] # 其余列

Please enter file name


In [3]:
num_features = features_set.shape[1]  # 特征的数量，即列数
num_instances = features_set.shape[0] # 实例的数量，即行数

print(f"This database has {num_features} features, with {num_instances} instances")

This database has 50 features, with 5000 instances


In [4]:
algorithm_name = 1
while True:
    print("Please type the number of algorithm:\n1: Forward Selection\n2: Backward Elimination\n")
    algorithm_name = input()
    
    try:
        algorithm_name = int(algorithm_name)
        
        if algorithm_name == 1 or algorithm_name == 2:
            break
        else:
            print("Invalid selection. Please enter 1 or 2.\n")
    except ValueError:
        print("Invalid input. Please enter a number (1 or 2).\n")

Please type the number of algorithm:
1: Forward Selection
2: Backward Elimination



In [7]:
# # 数据标准化/归一化
# scaler = StandardScaler()
# features_set_scaled = scaler.fit_transform(features_set)

In [8]:
# x_train, x_test, y_train, y_test = train_test_split(features_set_scaled, label_set, test_size = 0.3, random_state = 42)
# x_original = features_set_scaled
# x is feature set
# y is label set
x_original = features_set
y_original = label_set

# NN Algorithm

In [9]:
# Calculate euclidean distance
def euclidean_distance(instance1, instance2):
    # 计算两个实例对应特征差的平方和的平方根，即欧氏距离
    return np.sqrt(np.sum((instance1 - instance2) ** 2))

# 定义预测分类函数，使用最近邻算法进行分类
def predict_classification(x_train, y_train, x_test):
    # 初始化一个空列表，用于存储预测结果
    predictions = []
    # 遍历每一个测试实例
    for test_instance in x_test:
        # 计算测试实例与每一个训练实例之间的距离
        distances = [euclidean_distance(test_instance, x) for x in x_train]
        # 找到距离最近的训练实例的索引
        nearest_neighbor_index = np.argmin(distances)
        # 获取最近邻的类别标签
        predicted_class = y_train[nearest_neighbor_index]
        # 将预测结果添加到预测列表中
        predictions.append(predicted_class)
    # 返回所有测试实例的预测结果
    return predictions

# Calculate accuracy
# Check how y_predict match y_test
def accuracy_score(y_true, y_pred):
    # 计算预测正确的实例数
    correct = sum(y1 == y2 for y1, y2 in zip(y_true, y_pred))
    # 返回正确预测实例数占总实例数的比例，即准确率
    return correct / len(y_true)


# train, predict, and evaluate with leaving one out
def loocv_knn(X, y):
    y_true, y_pred = [], []
    
    for i in range(len(X)):
        # 测试集为第 i 个样本
        x_test = X[i:i+1]
        # 训练集为其余样本
        x_train = np.concatenate((X[:i], X[i+1:]), axis=0)

        # 测试集标签为第 i 个样本的标签
        y_test = y[i:i+1]
        # 训练集标签为其余样本的标签
        y_train = np.concatenate((y[:i], y[i+1:]), axis=0)
        
        # predict
        predicted_class = predict_classification(x_train, y_train, x_test)
        y_true.append(y_test[0])
        y_pred.append(predicted_class[0])
    
    # evaluate
    accuracy = accuracy_score(np.array(y_true), np.array(y_pred))
    
    return accuracy


In [10]:
# train, predict, and evaluate
accuracy = loocv_knn(x_original, y_original)

print(f'Accuracy: {accuracy}')

Accuracy: 0.6814


# Forward Selection

In [11]:
# 前向选择算法
def forward_selection(X, Y):
    # 初始化
    # n_features = num_features # 获取特征的数量
    selected_features = []
    remaining_features = list(range(num_features))  # 初始化剩余特征集为所有特征的索引列表
    best_score = 0

    while remaining_features:
    # 当剩余特征集不为空时
        # 初始化评分列表
        scores = []

        print(f"\nCurrent selected feature: {selected_features}")

        for feature in remaining_features:
            # 当前特征子集为已选择特征加上当前特征
            current_features = selected_features + [feature]
            # 获取训练集中特定特征子集的数据
            X_subset = X[:, current_features]
            # # 获取测试集中特定特征子集的数据
            # x_test_subset = x_test[:, current_features]
            
            # # 使用k-NN进行预测
            # y_pred = predict_classification(x_train_subset, y_train, x_test_subset, k)
            # # 计算预测的准确率
            # score = accuracy_score(y_test, y_pred)

            score = loocv_knn(X_subset, Y)
            

            print(f"\tCurrent feature: {current_features}; Current score: {score}")

            # 将当前特征的评分和特征索引添加到评分列表中
            scores.append((score, feature))
        
        # 按评分从高到低对特征进行排序
        scores.sort(reverse=True, key=lambda x: x[0])
        # 获取评分最高的特征及其评分
        current_best_score, current_best_feature = scores[0]

        if current_best_score > best_score:
        # 如果最佳评分大于0
            print(f'\t\tCurrent_best_feature: {current_best_feature}; Current_Best_score: {current_best_score}\n\n')

            best_score = current_best_score
            # 将评分最高的特征加入已选择特征集
            selected_features.append(current_best_feature)
            # 从剩余特征集中移除该特征
            remaining_features.remove(current_best_feature)

        else:
        # 如果最佳评分不大于0，停止选择过程
            break  

    print(f'\n\nbest_score: {best_score}')

    return selected_features, best_score

In [12]:
# 进行前向选择
selected_features, best_score = forward_selection(x_original, y_original)
num_selected_features = len(selected_features)

print(f"Forward_selected_features: {selected_features}")
print(f"Num_forward_selected_features: {num_selected_features}")


Current selected feature: []
	Current feature: [0]; Current score: 0.6814
	Current feature: [1]; Current score: 0.6964
	Current feature: [2]; Current score: 0.6942
	Current feature: [3]; Current score: 0.6814
	Current feature: [4]; Current score: 0.6872
	Current feature: [5]; Current score: 0.8388
	Current feature: [6]; Current score: 0.6964
	Current feature: [7]; Current score: 0.6916
	Current feature: [8]; Current score: 0.6846
	Current feature: [9]; Current score: 0.6926
	Current feature: [10]; Current score: 0.6826
	Current feature: [11]; Current score: 0.6976
	Current feature: [12]; Current score: 0.686
	Current feature: [13]; Current score: 0.6834
	Current feature: [14]; Current score: 0.6916
	Current feature: [15]; Current score: 0.6876
	Current feature: [16]; Current score: 0.6842
	Current feature: [17]; Current score: 0.6852
	Current feature: [18]; Current score: 0.6898
	Current feature: [19]; Current score: 0.6848
	Current feature: [20]; Current score: 0.695
	Current feature

# Backward Elimination

In [13]:
# 后向消除算法
def backward_elimination(x, y):
    # 初始化
    selected_features = list(range(num_features))
    best_accuracy = loocv_knn(x[:, selected_features], y) # 初始准确率

    
    while len(selected_features) > 1:
    # 当选择的特征数量大于1时，继续进行后向消除
        # 初始化
        best_feature_to_remove = None

        print(f"\nCurrent selected feature: {selected_features}")
        
        for feature in selected_features:
            # 创建当前特征集合，排除当前遍历的特征
            current_features = [f for f in selected_features if f != feature]
            # 计算移除该特征后的模型准确率
            accuracy = loocv_knn(x[:, current_features], y)

            print(f"\tCurrent removed feature: {feature}; Current selected features: {current_features}; Current score: {accuracy}")
            
            # 如果移除该特征后的模型准确率>=当前最佳准确率
            if accuracy >= best_accuracy:
                # 更新最佳准确率
                best_accuracy = accuracy
                # 标记当前特征为要移除的最佳特征
                best_feature_to_remove = feature
        
        # 如果找到了要移除的最佳特征
        if best_feature_to_remove is not None:
            # 从选择的特征集合中移除该特征
            selected_features.remove(best_feature_to_remove)

            print(f'\t\tRemoved feature: {best_feature_to_remove}; Selected features: {selected_features}; Current_Best_score: {best_accuracy}\n\n')

        else:
            # 如果没有找到要移除的特征，终止循环
            break

    print(f'\n\nbest_score: {best_accuracy}')
    return selected_features, best_accuracy

In [14]:
# 运行后向消除
selected_features, best_accuracy = backward_elimination(x_original, y_original)
num_selected_features = len(selected_features)
print(f'Selected Features: {selected_features}')
print(f"Num_forward_selected_features: {num_selected_features}")


Current selected feature: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
	Current removed feature: 0; Current selected features: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]; Current score: 0.6796
	Current removed feature: 1; Current selected features: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]; Current score: 0.6832
	Current removed feature: 2; Current selected features: [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]; Cur